# 15-Puzzle with A* Algorithm, BFS and Heuristics

This Python code implements a solver for the 8-puzzle problem using the A* search algorithm with different heuristic functions. It consists of several classes to represent the puzzle, the game, heuristics, and the agent.

## State Class

The `State` class represents a state of the 8-puzzle. It includes:
- `puzzle`: a 2D list representing the current puzzle configuration.
- `parent`: the parent state from which this state was reached.
- `action`: the action that led from the parent state to this state.
- `cost`: the cost (typically number of steps) to reach this state.

## Game Class

The `Game` class manages the 8-puzzle itself. It provides methods to:
- Check if a given state is the goal state.
- Find possible actions (e.g., 'up', 'down', 'left', 'right') for a given state.
- Generate the next state based on an action.
- Define the cost associated with taking an action from a state.
- Find the position of the blank (empty) tile in the puzzle.

## Heuristics Class

The `Heuristics` class contains two heuristic functions:
- `manhattan_distance`: Calculates the Manhattan distance heuristic for a given state.
    #####
    $\sum_{i=1}^{N} |v1[i] - v2[i]|$

    Where:

    - (`N`) is the upper limit of the summation, i.e., the number of elements you want to sum.
    - (`v1[i]`) represents the \(i\)-th element of vector `v1`.
    - (`v2[i]`) represents the \(i\)-th element of vector `v2`.
    #####
- `misplaced_tiles`: Calculates the number of misplaced tiles heuristic for a given state.
    #####
    $h_{\text{misplaced}}(s) = \sum_{i=1}^{N} \sum_{j=1}^{N}
    \begin{cases}
    1, & \text{if the tile at position }(i, j)\text{ is misplaced} \\
    0, & \text{otherwise}
    \end{cases}$

    Where:

    - (`misplaced(s)`) represents the number of misplaced tiles in the state \(s\).
    - (`N`) is the dimension of the puzzle (e.g., 8 for an 8-Puzzle).
    - The nested summations iterate over each row (`i`) and column (`j`) of the puzzle.
    - Last expression checks if the tile at position (`(i, j)`) is misplaced, and it adds 1 to the count if it is, or 0 if it is not.

## Agent Class

The `Agent` class is responsible for solving the 15-puzzle using different algorithms and heuristic functions. It is initialized with a game and a chosen heuristic function.
The `solve_BFS` method applies the Breadth-First Search (BFS) algorithm to find a solution, while the `solve_Astar_manhattan` method applies the A* algorithm with the Manhattan distance heuristic to find a solution. Both methods return the solution sequence as a list of states.

In [34]:
from collections import deque
import heapq
import random

class State:
    def __init__(self, puzzle, parent=None, action=None, cost=0):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.cost = cost
        self.manhattan = Heuristics.manhattan_distance(self)

    def __lt__(self, other):
        return (self.cost + self.manhattan) < (other.cost + other.manhattan)

class Game:
    def is_goal(self, state):
        goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
        return state.puzzle == goal_state

    def get_possible_actions(self, state):
        actions = []
        blank_x, blank_y = self.find_blank(state.puzzle)
        if blank_x > 0:
            actions.append("down")
        if blank_x < 3:
            actions.append("up")
        if blank_y > 0:
            actions.append("right")
        if blank_y < 3:
            actions.append("left")
        return actions

    def get_next_state(self, state, action):
        blank_x, blank_y = self.find_blank(state.puzzle)
        new_puzzle = [list(row) for row in state.puzzle]
        if action == "up":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x + 1][blank_y] = new_puzzle[blank_x + 1][blank_y], new_puzzle[blank_x][blank_y]
        elif action == "down":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x - 1][blank_y] = new_puzzle[blank_x - 1][blank_y], new_puzzle[blank_x][blank_y]
        elif action == "left":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x][blank_y + 1] = new_puzzle[blank_x][blank_y + 1], new_puzzle[blank_x][blank_y]
        elif action == "right":
            new_puzzle[blank_x][blank_y], new_puzzle[blank_x][blank_y - 1] = new_puzzle[blank_x][blank_y - 1], new_puzzle[blank_x][blank_y]
        return State(new_puzzle, state, action, state.cost + 1)

    def find_blank(self, puzzle):
        for i in range(4):
            for j in range(4):
                if puzzle[i][j] == 0:
                    return i, j

class Heuristics:
    @staticmethod
    def manhattan_distance(state):
        total_dist = 0
        for i in range(4):
            for j in range(4):
                if state.puzzle[i][j] != 0:
                    target_x, target_y = divmod(state.puzzle[i][j] - 1, 4)
                    total_dist += abs(i - target_x) + abs(j - target_y)
        return total_dist

    @staticmethod
    def misplaced_tiles(state):
        count = 0
        for i in range(4):
            for j in range(4):
                if state.puzzle[i][j] != goal_state[i][j]:
                    count += 1
        return count

class Agent:
    def __init__(self, game):
        self.game = game

    def solve_BFS(self, initial_state):
        # 1: Goal reached 2: No possible Actions 3: No solution
        queue = deque([initial_state])
        visited_states = set()
        level = 1
        nodes_explored = 0
        BFS_data = []

        while queue:
            level_size = len(queue)
            BFS_data.append([level, level_size])  # Stampa il livello corrente
            #print("Number of Nodes in this Level:", level_size)  # Stampa il numero di nodi in questo livello

            for _ in range(level_size):
                current_state = queue.popleft()
                current_state_tuple = tuple(map(tuple, current_state.puzzle))

                if current_state_tuple in visited_states:
                    continue  # Stato già visitato, salta questo stato

                visited_states.add(current_state_tuple)
                nodes_explored += 1

                if self.game.is_goal(current_state):
                    return BFS_data # Goal raggiunto, termina la ricerca

                possible_actions = self.game.get_possible_actions(current_state)
                if not possible_actions:
                    return  # No Possible Actions, Exiting

                for action in possible_actions:
                    next_state = self.game.get_next_state(current_state, action)
                    next_state_tuple = tuple(map(tuple, next_state.puzzle))
                    if next_state_tuple not in visited_states:
                        queue.append(next_state)

            level += 1

        return # No Solution Found

    def solve_Astar_manhattan(self, initial_state):
        visited = set()
        steps_data = []
        priority_queue = [(0, initial_state)]

        # Aggiungi il passo iniziale (stato iniziale)
        steps_data.insert(0, {
            "manhattan_distance": Heuristics.manhattan_distance(initial_state),
            "state": initial_state.puzzle,
            "action": None,
            "parent": None,
            "moves": 0
        })

        while priority_queue:
            current_cost, current_state = heapq.heappop(priority_queue)

            visited.add(tuple(map(tuple, current_state.puzzle)))

            if self.game.is_goal(current_state):
                return steps_data  # Aggiungi i dati solo quando il goal è raggiunto

            for action in self.game.get_possible_actions(current_state):
                next_state = self.game.get_next_state(current_state, action)
                next_state_tuple = tuple(map(tuple, next_state.puzzle))

                if next_state_tuple not in visited:
                    next_manhattan = Heuristics.manhattan_distance(next_state)
                    next_cost = current_state.cost + 1

                    heapq.heappush(priority_queue, (next_cost + next_manhattan, next_state))
                    # Aggiungi i dati qui per ogni stato esplorato
                    steps_data.append({
                        "manhattan_distance": next_manhattan,
                        "state": next_state.puzzle,
                        "action": action,
                        "parent": current_state.puzzle,
                        "moves": next_cost
                    })

        return None

    def solve_Astar_misplaced(self, initial_state):
        priority_queue = [(Heuristics.misplaced_tiles(initial_state), initial_state)]
        visited_states = set()
        steps_data = []

        while priority_queue:
            _, current_state = heapq.heappop(priority_queue)
            current_state_tuple = tuple(map(tuple, current_state.puzzle))

            if current_state_tuple in visited_states:
                continue  # Stato già visitato, salta questo stato

            visited_states.add(current_state_tuple)

            if self.game.is_goal(current_state):
                return steps_data  # Aggiungi i dati solo quando il goal è raggiunto

            count = Heuristics.misplaced_tiles(current_state)

            if not steps_data or count < steps_data[-1]["misplaced_tiles"]:
                steps_data.append({
                    "misplaced_tiles": count,
                    "state": current_state.puzzle,
                    "action": None,
                    "parent": current_state.parent.puzzle if current_state.parent else None
                })

            possible_actions = self.game.get_possible_actions(current_state)
            if not possible_actions:
                return None  # Nessuna azione possibile, termina la ricerca

            for action in possible_actions:
                next_state = self.game.get_next_state(current_state, action)
                next_state_tuple = tuple(map(tuple, next_state.puzzle))

                cost = Heuristics.misplaced_tiles(next_state)
                cost += next_state.cost

                if next_state_tuple not in visited_states:
                    heapq.heappush(priority_queue, (cost, next_state))
                    # Aggiungi i dati qui per ogni stato esplorato
                    steps_data.append({
                        "misplaced_tiles": cost,
                        "state": next_state.puzzle,
                        "action": action,
                        "parent": current_state.puzzle
                    })

        # Aggiungi il passo iniziale (stato iniziale)
        steps_data.insert(0, {
            "misplaced_tiles": Heuristics.misplaced_tiles(initial_state),
            "state": initial_state.puzzle,
            "action": None,
            "parent": None
        })

        return None

# Main execution of 15-Puzzle game

This Python code provides a solution for the 15-puzzle using the A* search algorithm and BFS (Breadth-First Search). It also includes functions for generating a random puzzle configuration and displaying the results.
In particular we have:

- `randomise_4x4_matrix` function creates a random 15-puzzle configuration. It initially generates a list of numbers from 0 to 15, shuffles them randomly, and organizes them into a 4x4 matrix.

- `solve_puzzle` function solves the puzzle using the chosen algorithm (`search`) and an initial configuration (`initial_state`). Here's how it works:

    - It initializes a `Game` object to manage the game.
    - It initializes an `Agent` object to perform the solving.
    - Depending on the chosen search algorithm (`search`), it calls the corresponding method to solve the puzzle.
    - It returns the solution, which is a sequence of states as a list.

- `print_matrix` function displays a state matrix. It can be used to print the initial configuration or any other state.

- `reconstruct_path` function rebuilds the path from the found solution. It takes the solution as input and starts with the final state. Then, it traces back to the initial state by following parent links in the solution data.

- `initial_state` variable represents the initial puzzle configuration. You can choose to initialize it randomly using `randomise_4x4_matrix` or manually specify a configuration as a matrix.

### Puzzle Resolution

The code uses three algorithms to solve the puzzle:
- BFS (Breadth-First Search)
- A* with the Manhattan distance heuristic
- A* with the misplaced tiles heuristic

For each algorithm, the `solve_puzzle` function is called with the desired algorithm and the initial configuration. The solution, if found, is stored and printed, along with the number of nodes explored by the algorithm.

### Solution Verification

If a solution is found by all three algorithms (BFS, A* with Manhattan distance, A* with misplaced tiles), the results are printed. The number of nodes explored by each algorithm and the initial puzzle configuration are displayed.

Remember that a solution might not be found if the initial configuration has an odd number of inversions.


In [35]:
def randomise_4x4_matrix():
    values = list(range(16))
    random.shuffle(values)
    matrix = [values[i:i + 4] for i in range(0, 16, 4)]
    return matrix

def solve_puzzle(search, initial_state):
    game = Game()
    agent = Agent(game)
    if search == "astar_manhattan":
        solution = agent.solve_Astar_manhattan(State(initial_state))
    elif search == "astar_misplaced":
        solution = agent.solve_Astar_misplaced(State(initial_state))
    elif search == "BFS":
        solution = agent.solve_BFS(State(initial_state))
    return solution

def print_matrix(matrix, init=1):
    for state in matrix[init:]:
        if init == 0:
            print(" ".join(map(str, state)))
        else:
            for row in state:
                print(" ".join(map(str, row)))
            print()

def reconstruct_path(solution):
    path = [solution[-1]['state']]  # Inizia con lo stato finale
    parent_state = solution[-1]['parent']

    while parent_state:
        if parent_state == solution[0]['state']:
            path.insert(0, parent_state)  # Inserisci lo stato iniziale all'inizio del percorso
            break
        parent_index = None
        for i, step in enumerate(solution):
            if step['state'] == parent_state:
                parent_index = i
                break

        if parent_index is not None:
            path.insert(0, parent_state)  # Inserisce il padre all'inizio del percorso
            parent_state = solution[parent_index]['parent']
        else:
            break  # Uscire se non viene trovato il genitore

    return path

successful = 0
goal_state = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

#initial_state = randomise_4x4_matrix() #
initial_state = [[1, 2, 3, 4],
                     [5, 6, 7, 8],
                     [9, 10, 12, 15],
                     [13, 14, 11, 0]]

solution_bfs = solve_puzzle("BFS", initial_state)
solution_astar_manhattan = solve_puzzle("astar_manhattan", initial_state)
solution_astar_misplaced = solve_puzzle("astar_misplaced", initial_state)

last_manhattan = []
last_misplaced = []

if (
    solution_bfs is not None and len(solution_bfs) > 0 and
    solution_astar_manhattan is not None and
    solution_astar_misplaced is not None
):
    print("Initial State\n")
    print_matrix(initial_state, init=0)
    print()
    print("Number of nodes explored with Breadth-First Search (BFS)             :", sum(item[1] for item in solution_bfs))
    print("Number of nodes explored with A* with Manhattan Distance heuristic   :", len(solution_astar_manhattan) - 1)
    print("Number of nodes explored with A* with Misplaced Tiles heuristic      :", len(solution_astar_misplaced) - 1)
    print()

    last_manhattan = [solution_astar_manhattan, len(solution_astar_manhattan) - 1]
    last_misplaced = [solution_astar_misplaced, len(solution_astar_misplaced) - 1]

    successful += 1
else:
    print("No solution found.")

Initial State

1 2 3 4
5 6 7 8
9 10 12 15
13 14 11 0

Number of nodes explored with Breadth-First Search (BFS)             : 41
Number of nodes explored with A* with Manhattan Distance heuristic   : 9
Number of nodes explored with A* with Misplaced Tiles heuristic      : 12



### For each level of the BFS search tree, the number of explored nodes is as follows:

In [56]:
for level, nodes in solution_bfs:
    print(level, nodes)

1 1
2 2
3 4
4 10
5 24


### Reconstructs the optimal path from the starting state to the goal state for both the Manhattan distance and misplaced tiles heuristics. It then compares the lengths of these paths and checks if they are the same.

In [51]:
solution_astar_manhattan_reconstructed = reconstruct_path(solution_astar_manhattan)
solution_astar_misplaced_reconstructed = reconstruct_path(solution_astar_misplaced)

print("Minimal number of moves for reaching goal (manhattan, misplaced)")
print(str(len(solution_astar_manhattan_reconstructed)) + ", " + str(len(solution_astar_misplaced_reconstructed)))
print()
print("Are minimal for both heuristics?")
print(solution_astar_manhattan_reconstructed == solution_astar_manhattan_reconstructed)

Minimal number of moves for reaching goal (manhattan, misplaced)
5, 5

Are minimal for both heuristics?
True


### Printing each state evaluated by the Manhattan Heuristic during the search process

In [59]:
for index in solution_astar_misplaced:
    print(index)

{'misplaced_tiles': 3, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]], 'action': None, 'parent': None}
{'misplaced_tiles': 5, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]], 'action': 'down', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]]}
{'misplaced_tiles': 5, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 0, 11]], 'action': 'right', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]]}
{'misplaced_tiles': 4, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]], 'action': None, 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]]}
{'misplaced_tiles': 7, 'state': [[1, 2, 3, 4], [5, 6, 7, 0], [9, 10, 12, 8], [13, 14, 11, 15]], 'action': 'down', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]]}
{'misplaced_tiles': 5, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]], 'action': 'right', 'paren

### Printing each state evaluated by the Misplaced Tiles Heuristic during the search process

In [61]:
for index in solution_astar_manhattan:
    print(index)

{'manhattan_distance': 4, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]], 'action': None, 'parent': None, 'moves': 0}
{'manhattan_distance': 3, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]], 'action': 'down', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]], 'moves': 1}
{'manhattan_distance': 5, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 0, 11]], 'action': 'right', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]], 'moves': 1}
{'manhattan_distance': 4, 'state': [[1, 2, 3, 4], [5, 6, 7, 0], [9, 10, 12, 8], [13, 14, 11, 15]], 'action': 'down', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]], 'moves': 2}
{'manhattan_distance': 2, 'state': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]], 'action': 'right', 'parent': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]], 'moves': 2}
{'manhattan_distance': 3, 'state': [[1, 2, 

### Reconstructed Paths for A* Search

Here, we print the reconstructed paths for two A* searches, each using a different heuristic:
- First, we display the reconstructed path for A* using the Manhattan distance heuristic.
- Then, we show the reconstructed path for A* using the misplaced tiles heuristic.

These paths represent the sequence of states leading to the goal state.


In [62]:
for index in solution_astar_manhattan_reconstructed:
    print(index)

print()

for index in solution_astar_misplaced_reconstructed:
    print(index)

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]

[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 15], [13, 14, 11, 0]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 12, 0], [13, 14, 11, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 0, 12], [13, 14, 11, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 0, 15]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]]
